# Step 01 - Import Pakages

In [ ]:
import os
import caer
import keras 
import cv2 as cv
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

# Step 02- Data Preprocessing

### Getting data from kaggle

In [ ]:
! mkdir ~/.kaggle
! cp /content/kaggle.json ~/.kaggle

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d alexattia/the-simpsons-characters-dataset

In [ ]:
# ! unzip /content/the-simpsons-characters-dataset.zip

### Preprocessing 

In [ ]:
IMG_SIZE = (80, 80)
Channels = 1
CHAR_PATH = r"/content/simpsons_dataset"

In [ ]:
char_dict = {"name": [], "count": [] }
for char in os.listdir(CHAR_PATH):
  char_dict['name'].append(char)
  char_dict['count'].append(len(os.listdir(os.path.join(CHAR_PATH, char))))



In [ ]:
df = pd.DataFrame(data=char_dict)

In [ ]:
sort_img = df.sort_values('count', ascending=False)[:10]

In [ ]:
sort_img['name'].values

In [ ]:
charector = ['homer_simpson', 'ned_flanders', 'moe_szyslak', 'lisa_simpson',
       'bart_simpson', 'marge_simpson', 'krusty_the_clown',
       'principal_skinner', 'charles_montgomery_burns',
       'milhouse_van_houten']

In [ ]:
train = caer.preprocess_from_dir(CHAR_PATH, charector, channels=Channels, IMG_SIZE=IMG_SIZE, isShuffle=True)

In [ ]:
len(train)

In [ ]:
train.shape

In [ ]:
plt.figure(figsize=(4, 4))
plt.imshow(train[2][0])
plt.show()

In [ ]:
feature, labels = caer.sep_train(train, IMG_SIZE=IMG_SIZE)

In [ ]:
feature.shape

In [ ]:
feature.max()

In [ ]:
# from keras.utils import to_categorical
# # normalizing through caer
# featureSet = caer.normalize(feature)
# feature_label = to_categorical(labels, len(charector))

In [ ]:
# normalizing the image into 255 to 1.0
feature = feature / 255.0

In [ ]:
feature.max()

In [ ]:
plt.figure(figsize=(4, 4))
plt.imshow(feature[50].reshape(80, 80))
plt.show()

In [ ]:
x_train, x_test, y_train, y_test = caer.train_val_split(feature, labels, 0.2)

In [ ]:
x_train.shape, y_test.shape, x_test.shape, y_train.shape

# Step 03 - Creating Model and Evalutation

## Creating Model

In [ ]:
# create model
model = keras.models.Sequential()

In [ ]:
# adding cnn layer
model.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same', input_shape=(80, 80, 1)))

# adding maxpool layers
model.add(keras.layers.MaxPool2D(pool_size=(2, 2), strides=2))

# adding dropout layers
model.add(keras.layers.Dropout(0.2))

In [ ]:
# adding cnn layer
model.add(keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'))

# adding maxpool layers
model.add(keras.layers.MaxPool2D(pool_size=(2, 2), strides=2))

# adding dropout layers
model.add(keras.layers.Dropout(0.4))

In [ ]:
# adding cnn layer
model.add(keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'))

# adding maxpool layers
model.add(keras.layers.MaxPool2D(pool_size=(2, 2), strides=2))

# adding dropout layers
model.add(keras.layers.Dropout(0.5))

In [ ]:
# falling layers
model.add(keras.layers.Flatten())


In [ ]:
# addind dense and output layers
model.add(keras.layers.Dense(units=128, activation='relu'))

model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense(64, activation='relu'))

model.add(keras.layers.Dense(units=10, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
%%time
# training the model
history = model.fit(x_train, y_train, epochs=15, validation_data=(x_test, y_test))

## Model Evaluation

In [ ]:
loss, accuracy = model.evaluate(x_train, y_train)

In [ ]:
print("Model loss Score:", loss)
print("Model Training Score:", accuracy)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
pred = np.argmax(y_pred, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print("The Accuracy Score of Testing:", accuracy_score(y_test, pred))

In [ ]:
n = 61
plt.figure()
plt.title(charector[pred[n]])
plt.imshow(x_test[n].reshape(80, 80), cmap='gray')
plt.show()
print(charector[y_test[n]])